In [1]:
import csv

In [2]:
import time
import pandas as pd
from datetime import datetime

import bernielib as bl

bl.listSerialPorts()

['COM3', 'COM4']

In [3]:
ber = bl.robot()

In [4]:
ber.home()

# Importing protocol from csv

In [5]:
filename = './samplesheet.csv'

In [6]:
with open(filename, mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    content = list(csv_reader)

In [7]:
content[3]

OrderedDict([('Parameters', 'Beads initial volume'),
             ('Comments', 'uL. How much of the beads is present'),
             ('0', '1000'),
             ('1', ''),
             ('2', ''),
             ('3', ''),
             ('4', ''),
             ('5', ''),
             ('6', ''),
             ('7', ''),
             ('8', ''),
             ('9', ''),
             ('10', ''),
             ('11', ''),
             ('12', '')])

#### Get row from a parameter

In [8]:
def getRowWithParameter(content, desired_parameter):
    for row in content:
        current_parameter = row['Parameters']
        if current_parameter == desired_parameter:
            return row

In [9]:
getRowWithParameter(content, 'Sample tube type')['2']

'eppendorf'

In [10]:
getRowWithParameter(content, 'Beads tube type')['0']

'eppendorf'

#### What positions contain samples?

In [11]:
def positionsToPurify(content):
    positions_list = []
    row = getRowWithParameter(content, 'Initial sample volume')
    for position in range(12):
        if float(row[str(position)]) > 0:
            positions_list.append(position)
    return positions_list

In [12]:
positionsToPurify(content)

[0, 1, 2]

#### Getting lists of all properties

In [13]:
def sortProperties(settings):
    """
    Will return 2 lists: one for protocol-wide properties, another one for sample-specific properties
    """
    protocol_wide_parameter_list = []
    sample_specific_parameter_list = []
    for row in settings:
        current_parameter = row['Parameters']
        position_missing_value = False
        # going through the values
        for position in range(12):
            value = row[str(position)]
            if value == '' or value is None:
                position_missing_value = True
        if position_missing_value:
            protocol_wide_parameter_list.append(current_parameter)
        else:
            sample_specific_parameter_list.append(current_parameter)

    return protocol_wide_parameter_list, sample_specific_parameter_list

In [14]:
properties_protocol, properties_sample = sortProperties(content)

In [15]:
def returnProtocolParameter(settings, param):
    row = getRowWithParameter(settings, param)
    value = row['0']
    try:
        value = float(value)
    except:
        pass
    return value

In [16]:
def returnSampleParameter(settings, param, position):
    row = getRowWithParameter(settings, param)
    value = row[str(position)]
    try:
        value = float(value)
    except:
        pass
    return value    

In [17]:
properties_protocol

['Text line',
 'Perform initial checkup',
 'Tip box refilled',
 'Beads initial volume',
 'Beads tube type',
 'Beads tube rack',
 'Beads tube column',
 'Beads tube well',
 'Eluent tube type',
 'Eluent tube rack',
 'Eluent tube position',
 'Eluent volume',
 'Ethanol tube type',
 'Ethanol tube rack',
 'Ethanol tube position',
 'Ethanol volume',
 'Waste tube type',
 'Waste tube rack',
 'Waste tube position',
 'Waste volume',
 'DNA absorption time',
 'Times to mix while absorbing',
 'Beads pulling time after absorption',
 'First stage ethanol wash time',
 'Second stage ethanol wash time',
 'Remove extra liquid with a fresh tip',
 'Time to dry after ethanol wash',
 'Elution time',
 'Times to mix while eluting']

In [18]:
returnProtocolParameter(content, 'DNA absorption time')

5.0

In [19]:
properties_sample

['Sample tube type',
 'Initial sample volume',
 'DNA size cutoff',
 'Fraction',
 'Beads volume',
 'First stage ethanol wash volume',
 'Second stage ethanol wash volume',
 'Elution volume']

In [20]:
returnSampleParameter(content, 'Sample tube type', 2)

'eppendorf'

In [21]:
returnSampleParameter(content, 'Beads volume', 2)

17.0

# Functions for an actual protocol

## Initializing tubes and samples

In [22]:
def initSamples(robot, settings):
    initial_vol_list = []
    # Getting list of positions at which samples are placed
    samples_positions_list = positionsToPurify(settings)
    # Obtaining initial sample volume from settings
    for position in samples_positions_list:
        volume = returnSampleParameter(settings, 'Initial sample volume', position)
        initial_vol_list.append(volume)
    # Initializing sample instances
    samples_list = bl.createSamplesToPurifyList(robot, initial_vol_list)
    return samples_list

In [38]:
def initResultTubes(robot, settings):
    # Getting list of positions at which samples are placed
    samples_positions_list = positionsToPurify(settings)
    N_samples = len(samples_positions_list) # Number of samples
    # Initializing results tube instances
    result_list = bl.createPurifiedSamplesList(robot, N_samples)
    return result_list

In [24]:
def initReagents(robot, settings):
    """
    Return 4 instances of the reagent tubes: 
    for beads, waste, eluent, ethanol 80%; in this particular order.
    The parameters for the tubes (such as volume and location) are also defined according to the sample sheet.
    """
    
    # Beads tube settings
    beads_tube_type = returnProtocolParameter(settings, 'Beads tube type')
    beads_rack_name = returnProtocolParameter(settings, 'Beads tube rack')
    if beads_rack_name == 'samples':
        beads_rack = robot.samples_rack
    elif beads_rack_name == 'reagents':
        beads_rack = robot.reagents_rack
    else:
        print ("wrong Beads tube rack specified in the samplesheet file")
        return
    beads_col = int(returnProtocolParameter(settings, 'Beads tube column'))
    beads_row = int(returnProtocolParameter(settings, 'Beads tube well'))
    V_avail_beads = returnProtocolParameter(settings, 'Beads initial volume')
    
    # Initializing beads tube
    beads_tube = bl.createSample(beads_tube_type, 'beads', beads_rack, beads_col, beads_row, V_avail_beads)

    
    # -------------------
    # Waste tube settings
    waste_tube_type = returnProtocolParameter(settings, 'Waste tube type')
    waste_rack_name = returnProtocolParameter(settings, 'Waste tube rack')
    if waste_rack_name == 'reagents':
        waste_rack = robot.reagents_rack
    else:
        print ("wrong Waste tube rack specified in the samplesheet file")
        return
    waste_col = 0
    waste_row = int(returnProtocolParameter(settings, 'Waste tube position'))
    V_waste = returnProtocolParameter(settings, 'Waste volume')
    
    # Initializing waste tube
    waste_tube = bl.createSample(waste_tube_type, 'liquid_waste', waste_rack, waste_col, waste_row, V_waste)

    
    # -------------------
    # Eluent tube settings
    eluent_tube_type = returnProtocolParameter(settings, 'Eluent tube type')
    eluent_rack_name = returnProtocolParameter(settings, 'Eluent tube rack')
    if eluent_rack_name == 'reagents':
        eluent_rack = robot.reagents_rack
    else:
        print ("wrong Eluent tube rack specified in the samplesheet file")
        return
    eluent_col = 0
    eluent_row = int(returnProtocolParameter(settings, 'Eluent tube position'))
    V_avail_eluent = returnProtocolParameter(settings, 'Eluent volume')
    
    # Initializing eluent tube
    eluent_tube = bl.createSample(eluent_tube_type, 'eluent', eluent_rack, eluent_col, eluent_row, V_avail_eluent)

    
    # -------------------
    # Ethanol tube settings
    ethanol_tube_type = returnProtocolParameter(settings, 'Ethanol tube type')
    ethanol_rack_name = returnProtocolParameter(settings, 'Ethanol tube rack')
    if ethanol_rack_name == 'reagents':
        ethanol_rack = robot.reagents_rack
    else:
        print ("wrong Ethanol tube rack specified in the samplesheet file")
        return
    ethanol_col = 0
    ethanol_row = int(returnProtocolParameter(settings, 'Ethanol tube position'))
    V_avail_ethanol = returnProtocolParameter(settings, 'Ethanol volume')    
    
    ethanol80_tube = bl.createSample(ethanol_tube_type, 'EtOH80pct', ethanol_rack, ethanol_col, ethanol_row, V_avail_ethanol)
    
    return beads_tube, waste_tube, eluent_tube, ethanol80_tube

## Functions to calculate beads volume

In [25]:
def getBeadsVolume(robot, settings, position):
    # Importing all possible parameters
    beads_volume = returnSampleParameter(settings, 'Beads volume', position)
    beads_volume_fraction = returnSampleParameter(settings, 'Fraction', position)
    dna_size_cutoff = returnSampleParameter(settings, 'DNA size cutoff', position)
    init_sample_vol = returnSampleParameter(settings, 'Initial sample volume', position)
    
    # Deciding which one to use
    if beads_volume > 0:
        use_beads_volume = beads_volume
    elif beads_volume <= 0 and beads_volume_fraction > 0:
        # If the beads volumes are not explicitly provided, use the volume multiplier (fraction)
        use_beads_volume = init_sample_vol * beads_volume_fraction
    elif beads_volume <= 0 and beads_volume_fraction <= 0 and dna_size_cutoff > 0:
        # Approximation using the beads manufacturer data
        # Using if neither beads volume, nor beads volume multiplier are explicitly provided.
        # Getting polynome coefficients
        a, b, c = robot.getBeadsVolumeCoef()
        # Calculating volume multiplier (fraction)
        multiplier = a + b / dna_size_cutoff + c / dna_size_cutoff ** 2
        use_beads_volume = init_sample_vol * multiplier
    else:
        print("No beads volume provided")
        use_beads_volume = 0
    return use_beads_volume    

In [26]:
def getBeadsVolumesForAllSamples(robot, settings, positions_list):
    beads_vol_list = []
    for position in positions_list:
        v = getBeadsVolume(robot, settings, position)
        beads_vol_list.append(v)
    return beads_vol_list

In [27]:
getBeadsVolumesForAllSamples(ber, content, [0, 1, 2])

[40.0, 30.0, 17.0]

## Functions to get ethanol wash volumes from settings

In [28]:
def getWashVolume(settings, stage):
    vol_list = []
    samples_positions_list = positionsToPurify(settings)
    if stage == 1:
        parameter_name = 'First stage ethanol wash volume'
    elif stage == 2:
        parameter_name = 'Second stage ethanol wash volume'
    else:
        parameter_name = 'First stage ethanol wash volume'
    for position in samples_positions_list:
        volume = returnSampleParameter(settings, parameter_name, position)
        vol_list.append(volume)
    return vol_list

## Functions to get eluent volume from settings

In [29]:
def getEluentVolume(settings):
    samples_positions_list = positionsToPurify(settings)
    vol_list = []
    for position in samples_positions_list:
        volume = returnSampleParameter(settings, 'Elution volume', position)
        vol_list.append(volume)
    return vol_list

In [43]:
def calcBeadsVolumeToAdd(robot, samples_list, cutoff_list):
    v_beads_list = []
    for sample, dna_size in zip(samples_list, cutoff_list):
        v_beads = robot.calcBeadsVol(sample, dna_size)
        v_beads_list.append(v_beads)
    return v_beads_list

def waitAfterTimestamp(timestamp, delay):
    new_ts = time.time()
    while (new_ts - timestamp) < delay:
        time.sleep(1)
        new_ts = time.time()

def waitAndMixByScript(robot, sample, timestamp, delay, mix_script, tip_col, tip_row):
    new_ts = time.time()
    robot.pickUpNextTip()
    robot.move(z=50)
    while (new_ts - timestamp) < delay:
        robot.mixByScript(sample, mix_script)
        robot.move(z=50)
        new_ts = time.time()
    robot.dumpTipToPosition(tip_col, tip_row)

def mixManySamples(robot, samples_list, timestamp, delay, mix_cycle_number, mix_script):
    delay_between_mixes = delay / (mix_cycle_number + 1)
    # Waiting before the first mix.
    # Needed for the case when the mix cycle number = 0. In this case, 
    # no mixing will happen, just waiting.
    new_ts = time.time()
    time_spent_pipetting = new_ts - timestamp
    wait_time_until_next_mix = delay_between_mixes - time_spent_pipetting
    if wait_time_until_next_mix >= 0:
        time.sleep(wait_time_until_next_mix)    
    for cycle in range(int(mix_cycle_number)):
        for sample in samples_list:
            robot.pickUpNextTip()
            robot.move(z=50)    
            robot.mixByScript(sample, mix_script)
            robot.move(z=50)
            robot.dumpTipToWaste()
            
        timestamp = new_ts
        new_ts = time.time()
        time_spent_pipetting = new_ts - timestamp
        wait_time_until_next_mix = delay_between_mixes - time_spent_pipetting
        if wait_time_until_next_mix >= 0:
            time.sleep(wait_time_until_next_mix)
    
    
def addBeads(robot, sample, beads, v_beads, sample_mix_scenario, beads_mix_scenario, z_safe=50):
    robot.move(z=z_safe)
    #robot.pickUpNextTip()
    #robot.move(z=z_safe)
    # Mixing beads before experiment
    #robot.mixByScript(beads, beads_mix_scenario)
    # Transferring beads to sample
    robot.transferLiquid(beads, sample, v_beads)
    # Mixing sample with beads
    robot.mixByScript(sample, sample_mix_scenario)
    robot.move(z=z_safe)
    #robot.dumpTipToWaste()
    robot.move(z=z_safe)

def addBeadsToAll(robot, samples_list, v_beads_list, beads, sample_mix_scenario, beads_mix_scenario, used_tip_fate='waste'):
    robot.moveMagnetsAway(poweroff=True)
    robot.pickUpNextTip()
    robot.move(z=50)
    robot.mixByScript(beads, beads_mix_scenario)
    robot.move(z=50)
    
    for sample, v_beads in zip(samples_list, v_beads_list):
        robot.transferLiquid(beads, sample, v_beads, touch_wall=False)
    
    counter = 0
    for sample, v_beads in zip(samples_list, v_beads_list):
        if counter != 0:
            robot.move(z=50)
            robot.pickUpNextTip()
        
        robot.move(z=50)
        
        robot.mixByScript(sample, sample_mix_scenario)
            
        if counter == 0:
            timestamp = time.time()
        counter += 1
        
        robot.move(z=50)
        if used_tip_fate == 'waste':
            robot.dumpTipToWaste()
        elif used_tip_fate == 'back':
            robot.returnTipBack()
        else:
            print("Wrong tip fate provided. Dumping the tip to waste.")
            robot.dumpTipToWaste()
    
    return timestamp

def removeSupernatant(robot, sample, waste, z_safe=50, delay=0.5):
    robot.move(z=z_safe)        # Move to the safe heigth
    robot.pickUpNextTip()       # Picking up a tip
    robot.move(z=z_safe)        # Move to the safe heigth
    # Uptaking liquid
    v = eluate_tube.getVolume() # Volume in the tube with the mix of the eluted DNA and beads
    # Performing an actual liquid transfer:
    robot.transferLiquid(source=sample, 
                         destination=waste, 
                         volume=v, 
                         dry_tube=True,
                         safe_z=z_safe, 
                         delay=pipette_delay, 
                         source_tube_radius=source_tube_radius)
    robot.move(z=z_safe)        # Moving up so the tip does not hit anything
    robot.dumpTipToWaste()      # Discarding the tip
    robot.move(z=z_safe)        # Moving up so the tip does not hit anything

def removeSupernatantAllSamples(robot, samples_list, waste):
    counter = 0
    for sample in samples_list:
        removeSupernatant(robot, sample, waste)
        if counter == 0:
            sample_dried_timestamp = time.time()
        counter += 1
    return sample_dried_timestamp

def add80PctEthanol(robot, samples_list, ethanol, volume_list, z_safe=50):
    robot.pickUpNextTip()
    robot.move(z=z_safe)
    
    counter = 0
    for sample, volume in zip(samples_list, volume_list):
        robot.transferLiquid(ethanol, sample, volume, touch_wall=False)
        if counter == 0:
            ethanol_added_time = time.time()
        counter += 1
    
    robot.move(z=z_safe)
    robot.dumpTipToWaste()
    
    return ethanol_added_time

def elutionMix(robot, sample, volume, delay=0.5):
    z0 = robot._getTubeZBottom(sample)
    z_top = sample.getSampleTopAbsZ(added_length=robot._calcExtraLength())
    robot.movePipetteToVolume(0)
    robot.movePipetteToVolume(volume+5)
    robot.movePipetteToVolume(volume)
    robot.move(z=z0-0.5)
    robot.movePipetteToVolume(0)
    time.sleep(delay)
    # Washing steps, moving along the wall
    # 1
    z_curr = z_top + 24
    #z_curr = sample.calcAbsLiquidLevelFromVol(500, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    robot.moveAxisDelta('X', 3.0)
    robot.movePipetteToVolume(volume/4.0)
    time.sleep(delay/4.0)
    # 2
    z_curr = z_top + 28
    #z_curr = sample.calcAbsLiquidLevelFromVol(300, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    #robot.moveAxisDelta('X', -0.629)
    robot.movePipetteToVolume(2 * (volume/4.0))
    time.sleep(delay/4.0)
    # 3
    z_curr = z_top + 32
    #z_curr = sample.calcAbsLiquidLevelFromVol(150, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    robot.moveAxisDelta('X', -0.629)
    robot.movePipetteToVolume(3 * (volume/4.0))
    time.sleep(delay/4.0)
    # 4
    z_curr = z_top + 36
    #z_curr = sample.calcAbsLiquidLevelFromVol(150, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    robot.moveAxisDelta('X', -0.629)
    robot.movePipetteToVolume(volume)
    time.sleep(delay/4.0)
    
    x, y = sample.getCenterXY()
    robot.move(x=x, y=y)
    z_curr = sample.calcAbsLiquidLevelFromVol(volume+100, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    
    robot.movePipetteToVolume(volume+50)
    time.sleep(delay)
    robot.movePipetteToVolume(0)

def elutionMixLrgVol(robot, sample, volume, delay=0.5):
    z0 = robot._getTubeZBottom(sample)
    z_top = sample.getSampleTopAbsZ(added_length=robot._calcExtraLength())
    
    # Uptaking
    robot.movePipetteToVolume(0)
    robot.movePipetteToVolume(volume+5)
    robot.movePipetteToVolume(volume)
    robot.move(z=z0-0.8)
    robot.movePipetteToVolume(0)
    time.sleep(delay)
    
    # Ejecting liquid
    z_curr = sample.calcAbsLiquidLevelFromVol(1000, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    robot.moveAxisDelta('X', 3.4)
    robot.movePipetteToVolume(volume+50)
    z_curr = sample.calcAbsLiquidLevelFromVol(500, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    
    # To back position
    x, y = sample.getCenterXY()
    robot.move(x=x, y=y)
    
    robot.movePipetteToVolume(0)

#TODO: make safe_z a general property of the robot
def elute(robot, sample, eluent, volume, mix_delay=0.5, mix_times=5, safe_z=50):
    robot.moveMagnetsAway(poweroff=True)
    robot.pickUpNextTip()
    robot.move(z=safe_z)
    robot.transferLiquid(eluent, sample, volume)
    for i in range(mix_times):
        elutionMixLrgVol(robot, sample, volume)
        elutionMixLrgVol(robot, sample, volume)
        elutionMix(robot, sample, volume)
    elution_start_time = time.time()
    elutionMix(robot, sample, volume)
    elutionMix(robot, sample, volume)
    
    robot.move(z=safe_z)
    #robot.returnTipBack()
    robot.dumpTipToWaste()
    sample.setVolume(volume)
    return elution_start_time

def eluteAllSamples(robot, samples_list, eluent, V_eluent_list, mix_delay=0.5, mix_times=4, safe_z=50):
    counter = 0
    for sample, V_eluent in zip(samples_list, V_eluent_list):
        ts = elute(ber, sample, eluent, V_eluent, mix_delay=mix_delay, mix_times=mix_times, safe_z=safe_z)
        print ()
        if counter == 0:
            elution_start_timestamp = ts
        counter += 1
    return elution_start_timestamp

def separateEluate(robot, eluate_tube, result_tube, pipette_delay=0.5, source_tube_radius=2, safe_z=50):
    robot.move(z=safe_z)        # Moving up so the tip does not hit anything
    robot.pickUpNextTip()       # Getting a new tip
    robot.move(z=safe_z)        # Moving up so the tip does not hit anything
    # Uptaking liquid
    v = eluate_tube.getVolume()      # Volume in the tube with the mix of the eluted DNA and beads
    # Performing an actual liquid transfer:
    robot.transferLiquid(source=eluate_tube, 
                         destination=result_tube, 
                         volume=v, 
                         dry_tube=True,
                         safe_z=safe_z, 
                         delay=pipette_delay, 
                         source_tube_radius=source_tube_radius)
    robot.move(z=safe_z)        # Moving up so the tip does not hit anything
    robot.dumpTipToWaste()      # Discarding the tip
    robot.move(z=safe_z)        # Moving up so the tip does not hit anything

def separateEluateAllTubes(robot, eluate_list, results_list):
    for sample, result in zip(eluate_list, results_list):
        separateEluate(robot, sample, result)
    
    
def oneStagePurification(robot, settings, cutoff_list, 
                         sample_mix_scenario, beads_mix_scenario,
                         T_wash=30, T_dry=300, T_elute=600,
                         V_wash=200, V_elute=30):
    
    print("Loading parameters from the sample sheet. Started ", datetime.now().strftime("%H:%M:%S"))
    samples_list = initSamples(robot, settings)
    result_list = initResultTubes(robot, settings)
    beads, waste, water, EtOH80pct = initReagents(robot, settings)
    positions_list = positionsToPurify(settings)
    v_beads_list = getBeadsVolumesForAllSamples(robot, settings, positions_list)
    v_ethanol_1st_stage_list = getWashVolume(settings, 1)
    v_ethanol_2nd_stage_list = getWashVolume(settings, 2)
    v_eluent_list = getEluentVolume(settings)
    DNA_absorb_mix_number = int(returnProtocolParameter(settings, 'Times to mix while absorbing'))
    elution_mix_numbers = int(returnProtocolParameter(settings, 'Times to mix while eluting'))
    # All times are provided in miuntes in sample sheet (for user's convenience),
    # but the library uses seconds. Don't forget to multiply by 60.
    T_absorb = returnProtocolParameter(settings, 'DNA absorption time') * 60.0
    T_pull = returnProtocolParameter(settings, 'Beads pulling time after absorption') * 60.0
    T_wash_1 = returnProtocolParameter(settings, 'First stage ethanol wash time') * 60.0
    T_wash_2 = returnProtocolParameter(settings, 'Second stage ethanol wash time') * 60.0
    T_dry = returnProtocolParameter(settings, 'Time to dry after ethanol wash') * 60.0
    T_elute = returnProtocolParameter(settings, 'Elution time') * 60.0

    print("Experiment started ", datetime.now().strftime("%H:%M:%S"))    
    # Adding beads
    timestamp_beads_added = addBeadsToAll(robot, samples_list, v_beads_list, beads, 
                                          sample_mix_scenario, beads_mix_scenario, used_tip_fate='waste')
    print("Beads added ", datetime.now().strftime("%H:%M:%S"))
    print("Waiting for DNA absorption")
    mixManySamples(robot, samples_list, timestamp_beads_added, T_absorb, DNA_absorb_mix_number, sample_mix_scenario)
    print("DNA absorption finished ", datetime.now().strftime("%H:%M:%S"))
    # Removing supernatant
    robot.moveMagnetsTowardsTube(poweroff=True)
    time.sleep(T_pull)
    print("Beads pulled to the side ", datetime.now().strftime("%H:%M:%S"))
    ts = removeSupernatantAllSamples(robot, samples_list, waste, how='fast')
    print("Supernatant removed ", datetime.now().strftime("%H:%M:%S"))
    # Ethanol wash
    # Wash 1
    timestamp_ethanol_added = add80PctEthanol(robot, samples_list, EtOH80pct, v_ethanol_1st_stage_list)
    print("Wash 1: ethanol added ", datetime.now().strftime("%H:%M:%S"))
    waitAfterTimestamp(timestamp_ethanol_added, T_wash_1)
    print("Wash 1: ethanol incubation finished ", datetime.now().strftime("%H:%M:%S"))
    ts = removeSupernatantAllSamples(robot, samples_list, waste, how='fast')
    print("Wash 1: ethanol removed ", datetime.now().strftime("%H:%M:%S"))
    # Wash 2
    timestamp_ethanol_added = add80PctEthanol(robot, samples_list, EtOH80pct, v_ethanol_2nd_stage_list)
    print("Wash 2: ethanol added ", datetime.now().strftime("%H:%M:%S"))
    waitAfterTimestamp(timestamp_ethanol_added, T_wash_2)
    print("Wash 2: ethanol incubation finished ", datetime.now().strftime("%H:%M:%S"))
    timestamp_ethanol_removed = removeSupernatantAllSamples(robot, samples_list, waste, how='full')
    print("Wash 2: ethanol removed ", datetime.now().strftime("%H:%M:%S"))
    # Drying ethanol
    waitAfterTimestamp(timestamp_ethanol_removed, T_dry)
    print("Ethanol drying finished ", datetime.now().strftime("%H:%M:%S"))
    
    # Elution
    # Adding water
    elution_start_timestamp = eluteAllSamples(robot, samples_list, water, v_eluent_list, mix_delay=0.5, mix_times=6, safe_z=50)
    print("Eluent added ", datetime.now().strftime("%H:%M:%S"))
    mixManySamples(robot, samples_list, elution_start_timestamp, T_elute, elution_mix_numbers, sample_mix_scenario)
    print("Elution incubation finished ", datetime.now().strftime("%H:%M:%S"))
    
    # Magnetic beads to the side of the tubes
    ber.moveMagnetsTowardsTube()
    time.sleep(T_pull)
    print("Beads pulled to the side ", datetime.now().strftime("%H:%M:%S"))
    # Moving liquid to the resulting tubes
    separateEluateAllTubes(robot, samples_list, result_list)
    print("Eluate transferred to the new tube ", datetime.now().strftime("%H:%M:%S"))
    print("Experiment finished ", datetime.now().strftime("%H:%M:%S"))

In [31]:
mix_script = pd.read_csv('mixing_pattern_eppendorf.csv')

#### Preparing fake samples for the experiments

In [33]:
water = bl.createSample('25ml', 'water', ber.reagents_rack, 0, 0, 25000)
sample1 = bl.createSample('eppendorf', 's1', ber.samples_rack, 1, 0, 30)
sample2 = bl.createSample('eppendorf', 's2', ber.samples_rack, 1, 1, 30)
sample3 = bl.createSample('eppendorf', 's3', ber.samples_rack, 1, 2, 30)

In [34]:
ber.pickUpNextTip()
ber.transferLiquid(water, sample1, 30)
ber.transferLiquid(water, sample2, 30)
ber.transferLiquid(water, sample3, 30)
ber.move(z=50)
ber.dumpTipToWaste()

#### Testing protocol functions

In [35]:
with open(filename, mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    content = list(csv_reader)

In [36]:
oneStagePurification(ber, content, cutoff_list=[150], sample_mix_scenario=mix_script, beads_mix_scenario=mix_script)

Loading parameters from the sample sheet. Started  16:14:40
Experiment started  16:14:40
Beads added  16:17:31
Waiting for DNA absorption
DNA absorption finished  16:20:14
Beads pulled to the side  16:21:46
Supernatant removed  16:22:24
Wash 1: ethanol added  16:22:49
Wash 1: ethanol incubation finished  16:23:04
Wash 1: ethanol removed  16:23:41
Wash 2: ethanol added  16:24:07
Wash 2: ethanol incubation finished  16:24:21
Wash 2: ethanol removed  16:25:06
Ethanol drying finished  16:28:36



Eluent added  16:32:25
Elution incubation finished  16:34:55
Beads pulled to the side  16:36:27


TypeError: zip argument #2 must support iteration

In [73]:
ber.data

{'magnets_away_angle': 5.2,
 'magnets_near_tube_angle': 11.2,
 'stair_finding_step_list': [1, 0.2],
 'stair_finding_z_increment': 0.1,
 'stair_finding_z_retract_after_trigger': -1,
 'stair_finding_z_max_travel': 3,
 'stair_finding_z_load_threshold': 500,
 'z_max': 180,
 'x_max': 189,
 'y_max': 322,
 'added_tip_length': 41.6,
 'volume_to_position_slope': -0.15859245180518214,
 'volume_to_position_intercept': -0.958195131933648,
 'pipetting_delay': 0.2,
 'DNAsize_to_Vbeads': {'a': 0.499325349, 'b': -9.91043764, 'c': 25758.5836},
 'tip_drop_servo_up_angle': 2.5,
 'tip_drop_servo_down_angle': 7.3,
 'speed_XY': 50000,
 'speed_Z': 30000,
 'speed_pipette': 2500,
 'plunger_movement_when_dumping_tip': 35}

#### Testing separateEluate function

In [40]:
samples_list = initSamples(ber, content)
result_list = initResultTubes(ber, content)

In [41]:
for sample, result in zip(samples_list, result_list):
    separateEluate(ber, sample, result)

#### Removing liquids from sample tubes (after a failed attempt)

In [44]:
waste = bl.createSample('25ml', 'liquid_waste', ber.reagents_rack, 0, 1, 0)
sample1 = bl.createSample('eppendorf', 's1', ber.samples_rack, 1, 0, 30)
sample2 = bl.createSample('eppendorf', 's2', ber.samples_rack, 1, 1, 30)
sample3 = bl.createSample('eppendorf', 's3', ber.samples_rack, 1, 2, 30)

In [36]:
ber.pickUpNextTip()
ber.transferLiquid(sample1, waste, 200)
ber.transferLiquid(sample2, waste, 200)
ber.transferLiquid(sample3, waste, 200)
ber.move(z=50)
ber.dumpTipToWaste()

#### Refilling the tip rack

In [44]:
ber.tips_rack.refill()

In [45]:
sample1._allowPlungerLagCompensation(100, 5)

0

# Testing transferLiquid for elution

In [5]:
water = bl.createSample('25ml', 'water', ber.reagents_rack, 0, 0, 25000)
waste = bl.createSample('25ml', 'liquid_waste', ber.reagents_rack, 0, 1, 0)
sample1 = bl.createSample('eppendorf', 's1', ber.samples_rack, 1, 0, 0)

In [9]:
result1 = bl.createSample('eppendorf', 'r1', ber.samples_rack, 0, 0, 0)

## Removing any remaining liquid

In [6]:
ber.move(z=50)
ber.pickUpNextTip()
ber.move(z=50)
ber.transferLiquid(sample1, waste, 199, dry_tube=True)
ber.move(z=50)
ber.dumpTipToWaste()
ber.move(z=50)

AttributeError: 'robot' object has no attribute 'dropTipToWaste'

In [7]:
ber.dumpTipToWaste()

## Adding liquid to the tube (preparing for the test)

In [11]:
ber.move(z=50)
ber.pickUpNextTip()
ber.move(z=50)
ber.transferLiquid(water, sample1, 30)
ber.move(z=50)
ber.dumpTipToWaste()
ber.move(z=50)

## Testing!

In [12]:
ber.move(z=50)
ber.pickUpNextTip()
ber.move(z=50)
ber.transferLiquid(sample1, result1, 30, dry_tube=True)
ber.move(z=50)
ber.dumpTipToWaste()
ber.move(z=50)

## Adding and removing lots of liquid

In [13]:
ber.move(z=50)
ber.pickUpNextTip()
ber.move(z=50)
ber.transferLiquid(water, sample1, 530)
ber.move(z=50)
ber.dumpTipToWaste()
ber.move(z=50)

In [14]:
ber.move(z=50)
ber.pickUpNextTip()
ber.move(z=50)
ber.transferLiquid(sample1, result1, 530, dry_tube=True)
ber.move(z=50)
ber.dumpTipToWaste()
ber.move(z=50)

# Testing purify_to_cutoff script

In [1]:
import importlib
import purify_one_cutoff as ponec
import time

In [19]:
ponec = importlib.reload(ponec)

In [12]:
print("Testing mixManySamples function.")
# Preparing for the test
settings = ponec.loadSettings('samplesheet.csv')
ber = ponec.bl.robot()
ber.home()
samples_list = ponec.initSamples(ber, settings)
for sample in samples_list:   # For the test, sample starts with zero volume
    sample.setVolume(400)
beads, waste, water, EtOH80pct = ponec.initReagents(ber, settings)
# Adding the liquid
#ber.pickUpNextTip()
#ber.move(z=50)
#for sample in samples_list:
#    ber.transferLiquid(water, sample, 100)
#ber.move(z=50)
#ber.dumpTipToWaste()
#ber.move(z=50)

timestamp = time.time()

# Performing an actual test
ponec.mixManySamples(ber, samples_list, timestamp, settings)

# Cleaning update
ber.pickUpNextTip()
ber.move(z=50)
for sample in samples_list:
    ber.transferLiquid(sample, waste, 400, dry_tube=True)
ber.move(z=50)
ber.dumpTipToWaste()
ber.move(z=50)
ber.close()

print("Test finished.")
print("Robot should have attempted to wait for some time, mix samples, then wait again.")
val = input("Did it happened? Enter y if yes, anything else if no: ")
if val == 'y':
    print("mixManySamples success.")
else:
    print("mixManySamples failed.")
    sys.exit("Test failed. Terminating.")
    


Testing mixManySamples function.
Test finished.
Robot should have attempted to wait for some time, mix samples, then wait again.
Did it happened? Enter y if yes, anything else if no: y
mixManySamples success.


In [6]:
ber.close()

In [3]:
ponec = importlib.reload(ponec)

In [2]:
print("Testing addBeadsToAll function")
# Preparing for the test
settings = ponec.loadSettings('samplesheet.csv')
ber = ponec.bl.robot()
ber.home()
samples_list = ponec.initSamples(ber, settings)
for sample in samples_list:   # For the test, sample starts with zero volume
    sample.setVolume(0)
beads, waste, water, EtOH80pct = ponec.initReagents(ber, settings)
positions_list = ponec.positionsToPurify(settings)
v_beads_list = ponec.getBeadsVolumesForAllSamples(ber, settings, positions_list)

# Adding the initial liquid
ber.pickUpNextTip()
ber.move(z=50)
for sample in samples_list:
    ber.transferLiquid(water, sample, 30)
ber.move(z=50)
ber.dumpTipToWaste()
ber.move(z=50)

# Performing an actual test
ponec.addBeadsToAll(ber, samples_list, v_beads_list, beads)

# Cleaning update
ber.move(z=50)
ber.pickUpNextTip()
ber.move(z=50)
for sample in samples_list:
    v = sample.getVolume()
    ber.transferLiquid(sample, waste, v, dry_tube=True)
ber.move(z=50)
ber.dumpTipToWaste()
ber.move(z=50)
ber.close()

print("Test finished.")
print("Robot should have mixed the beads tube, transfer beads to the samples, mix each sample.")
val = input("Did it happened? Enter y if yes, anything else if no: ")
if val == 'y':
    print("addBeadsToAll success.")
else:
    print("addBeadsToAll failed.")
    sys.exit("Test failed. Terminating.")

Testing addBeadsToAll function
Sample at position 0 will receive 40.0 uL of magnetic beads.
Sample at position 1 will receive 30.0 uL of magnetic beads.
Sample at position 2 will receive 17.0 uL of magnetic beads.
Test finished.
Robot should have mixed the beads tube, transfer beads to the samples, mix each sample.
Did it happened? Enter y if yes, anything else if no: y
addBeadsToAll success.


In [17]:
print("Testing ethanol wash functions")
# Preparing for the test
settings = ponec.loadSettings('samplesheet.csv')
ber = ponec.bl.robot()
ber.home()
samples_list = ponec.initSamples(ber, settings)
for sample in samples_list:   # For the test, sample starts with zero volume
    sample.setVolume(0)
beads, waste, water, EtOH80pct = ponec.initReagents(ber, settings)
positions_list = ponec.positionsToPurify(settings)
v_ethanol_list = ponec.getWashVolume(settings, 1)

# Performing an actual test
# Adding ethanol
ponec.add80PctEthanol(ber, samples_list, EtOH80pct, v_ethanol_list)
# Removing supernatant
ponec.removeSupernatantAllSamples(ber, samples_list, waste)

# Cleaning
ber.close()

print("Test finished.")
print("Robot should have transferred ethanol to all tubes, then remove all ethanol.")
val = input("Did it happened? Enter y if yes, anything else if no: ")
if val == 'y':
    print("Ethanol wash success.")
else:
    print("Ethanol wash failed.")
    sys.exit("Test failed. Terminating.")

Testing ethanol wash functions
Test finished.
Robot should have transferred ethanol to all tubes, then remove all ethanol.
Did it happened? Enter y if yes, anything else if no: y
Ethanol wash success.


In [5]:
ber.close()

In [11]:
samples_list[0].stype.data

{'z_above_racks_dict': {'samples': 11.5},
 'inner_diameter': 9.15,
 'depth_to_vol_dict': {'0': 38.45,
  '100': 32,
  '200': 28,
  '300': 26,
  '400': 24,
  '500': 22,
  '700': 18,
  '1000': 13,
  '1500': 5.8,
  '1700': 3},
 'extra_immersion_volume': 200,
 'close_to_bottom_volume': 50,
 'low_vol_uptake_single_step': 0.5,
 'low_vol_uptake_number_of_steps': 3,
 'low_vol_uptake_delay_between_steps': 1,
 'mix_script_file_path': 'mixing_pattern_eppendorf.csv'}

In [6]:
ber.powerStepperOff('A') # to prevent unnecessary heating

In [6]:
print("Testing elute functions")
# Preparing for the test
settings = ponec.loadSettings('samplesheet.csv')
ber = ponec.bl.robot()
ber.home()
samples_list = ponec.initSamples(ber, settings)
result_list = ponec.initResultTubes(ber, settings)

for sample in samples_list:   # For the test, sample starts with zero volume
    sample.setVolume(0)
beads, waste, water, EtOH80pct = ponec.initReagents(ber, settings)
positions_list = ponec.positionsToPurify(settings)
v_water_list = ponec.getEluentVolume(settings)

# Performing an actual test
ponec.eluteAllSamples(ber, samples_list, water, v_water_list, settings)
ponec.separateEluateAllTubes(ber, samples_list, result_list)

ber.powerStepperOff('A') # to prevent unnecessary heating

ber.close()

print("Test finished.")
print("Robot should have put a small volume of a liquid, thoroughly mix, then pipette everything out.")
val = input("Did it happened? Enter y if yes, anything else if no: ")
if val == 'y':
    print("Elution success.")
else:
    print("Elution failed.")
    sys.exit("Test failed. Terminating.")

Testing elute functions



Test finished.
Robot should have put a small volume of a liquid, thoroughly mix, then pipette everything out.
Did it happened? Enter y if yes, anything else if no: y
Elution success.


# Testing entire 1-step purification

In [1]:
import importlib
import purify_one_cutoff as ponec
import time

In [2]:
ponec = importlib.reload(ponec)

In [3]:
print("Testing entire 1-step purification function")
settings = ponec.loadSettings('samplesheet.csv')
ber = ponec.bl.robot()
ber.home()

# initializing samples, so I can fill them with initial liquid
samples_list = ponec.initSamples(ber, settings)
for sample in samples_list:   # For the test, sample starts with zero volume
    sample.setVolume(0)
beads, waste, water, EtOH80pct = ponec.initReagents(ber, settings)
positions_list = ponec.positionsToPurify(settings)

# Refilling the tips
#ber.tips_rack.refill()

# Adding the initial liquid
ber.pickUpNextTip()
ber.move(z=50)
for sample in samples_list:
    ber.transferLiquid(water, sample, 30)
ber.move(z=50)
ber.dumpTipToWaste()
ber.move(z=50)

# Performing actual test
ponec.purify_one_cutoff(ber, settings)

print("Test finished.")

Testing entire 1-step purification function
Sample at position 0 will receive 40.0 uL of magnetic beads.
Sample at position 1 will receive 30.0 uL of magnetic beads.
Sample at position 2 will receive 17.0 uL of magnetic beads.
This is the one-stage magnetic beads purification.
It will remove any DNA and other molecules of low molecular weight.
I will purify 3 samples
I will use the following magnetic beads volumes to purify: 
[40.0, 30.0, 17.0]
I will elute the resulting DNA with eluent of the following volumes: 
[30.0, 30.0, 30.0]

Purification started  15:45:48

Adding magnetic beads. Started  15:45:48
Beads added  15:48:28
Now waiting for DNA absorption.
DNA absorption finished  15:51:12
Now pulling the beads towards the tube side
Pulling time is 1.5 minutes
Started  15:51:12
Beads are now on the side of the tube.
Removing supernatant. Started  15:52:43
Supernatant removed from all tubes at  15:53:39
Now starting ethanol washes.
Adding 80% ethanol. Started  15:53:39
Wash 1: ethanol 

In [4]:
ber.close()

# Testing two-step purification

In [1]:
import importlib
import purify_one_cutoff as ponec
import time

In [2]:
ponec = importlib.reload(ponec)

In [3]:
print("Testing entire 2-step purification function")
settings = ponec.loadSettings('samplesheet_2stages.csv')
ber = ponec.bl.robot()
ber.home()

# initializing samples, so I can fill them with initial liquid
samples_list = ponec.initSamples(ber, settings)
for sample in samples_list:   # For the test, sample starts with zero volume
    sample.setVolume(0)
beads, waste, water, EtOH80pct = ponec.initReagents(ber, settings)
positions_list = ponec.positionsToPurify(settings)

# Refilling the tips
#ber.tips_rack.refill()

# Adding the initial liquid
ber.pickUpNextTip()
ber.move(z=50)
for sample in samples_list:
    ber.transferLiquid(water, sample, 30)
ber.move(z=50)
ber.dumpTipToWaste()
ber.move(z=50)

# Performing actual test
ponec.purifyTwoCutoffs(ber, settings)

print("Test finished.")

Testing entire 2-step purification function
Sample at position 0 will receive 15.69954088270588 uL of magnetic beads at first stage.
Sample at position 1 will receive 15.957134089733332 uL of magnetic beads at first stage.
Sample at position 2 will receive 16.630787221333332 uL of magnetic beads at first stage.
Sample at position 0 will receive 40.0 uL of magnetic beads at the second stage.
Sample at position 1 will receive 30.0 uL of magnetic beads at the second stage.
Sample at position 2 will receive 17.0 uL of magnetic beads at the second stage.
This is the two-stages magnetic beads purification.
It will remove any DNA and other molecules of too low and too high molecular weight.
I will purify 3 samples
I will use the following magnetic beads volumes for short DNA cutoff: 
[15.69954088270588, 15.957134089733332, 16.630787221333332]
I will use the following magnetic beads volumes for long DNA cutoff: 
[40.0, 30.0, 17.0]
I will elute the resulting DNA with eluent of the following vol

In [4]:
ber.powerStepperOff('A') # to prevent unnecessary heating

ber.close()

In [9]:
ber.data

{'magnets_away_angle': 5.2,
 'magnets_near_tube_angle': 11.2,
 'stair_finding_step_list': [1, 0.2],
 'stair_finding_z_increment': 0.1,
 'stair_finding_z_retract_after_trigger': -1,
 'stair_finding_z_max_travel': 3,
 'stair_finding_z_load_threshold': 500,
 'z_max': 180,
 'x_max': 189,
 'y_max': 322,
 'added_tip_length': 41.6,
 'volume_to_position_slope': 0.12353871734335235,
 'volume_to_position_intercept': 0.34573859874557084,
 'pipetting_delay': 0.2,
 'DNAsize_to_Vbeads': {'a': 0.499325349, 'b': -9.91043764, 'c': 25758.5836},
 'tip_drop_servo_up_angle': 2.5,
 'tip_drop_servo_down_angle': 7.3,
 'speed_XY': 50000,
 'speed_Z': 30000,
 'speed_pipette': 2500,
 'plunger_movement_when_dumping_tip': 35,
 'maximum_plunger_movement_coordinate': 35,
 'liquid_uptake_low_volume_bottom_offset': 0.2,
 'position_to_volume_slope': 8.07539335665879,
 'position_to_volume_intercept': -2.55498251749027}

In [10]:
ber.pipetteServoDown()

In [16]:
ber.pipetteMove(20)

In [5]:
ber.dumpTip()

In [6]:
ber.pickUpNextTip()

(3, 7)

In [7]:
ber.move(z=50)

In [8]:
ber.dumpTipToWaste()